In [166]:
import tensorflow as tf
#import tensorflow_probability as tfp
import numpy as np
from tensorflow.keras import layers, Model
from tensorflow.keras import layers as L

class PhysicsInformedNN(Model):
    # Initialize the class
    def __init__(self, x0, u0, x1, u1, layers, dt, lb, ub, q):
        super(PhysicsInformedNN, self).__init__()

        self.lb = lb
        self.ub = ub

        self.x0 = tf.cast(x0, dtype=tf.float32)
        self.x1 = tf.cast(x1, dtype=tf.float32)

        self.u0 = tf.cast(u0, dtype=tf.float32)
        self.u1 = tf.cast(u1, dtype = tf.float32)

        self.nn_layers = layers
        self.dt = dt
        self.q = max(q,1)

        # Initialize NN
        self.model = self.initialize_NN(self.nn_layers)

        # Initialize parameters
        self.lambda_1 = tf.Variable([0.0], dtype=tf.float32)
        self.lambda_2 = tf.Variable([-6.0], dtype=tf.float32)

        # Load IRK weights
        tmp = np.float32(np.loadtxt('./PINNs-master/Utilities/IRK_weights/Butcher_IRK%d.txt' % (q), ndmin = 2))
        weights =  np.reshape(tmp[0:q**2+q], (q+1,q))
        self.IRK_alpha = weights[0:-1,:]
        self.IRK_beta = weights[-1:,:]
        self.IRK_times = tmp[q**2+q:]

        self.dummy_x0_tf = tf.ones((self.x0.shape[0], self.q)) # dummy variable for fwd_gradients
        self.dummy_x1_tf = tf.ones((self.x1.shape[0], self.q)) # dummy variable for fwd_gradients

        self.dummy_x0_tf2 = tf.ones((self.x0.shape[0], self.nn_layers[-2])) # dummy variable for second gradient
        self.dummy_x1_tf2 = tf.ones((self.x1.shape[0], self.nn_layers[-2])) # dummy variable for second gradient


    def initialize_NN(self, layers):        
        model = tf.keras.models.Sequential()
        for l in range(0,len(layers)-2):
            model.add(L.Dense(units=layers[l+1], activation='tanh', input_dim=layers[l]))
        model.add(L.Dense(units=layers[-1]))
        return model

    @tf.function
    def net_U0(self, x):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            U = self.model(x)
            print(f"U: {U}")
            U_x = tape.gradient(U, x)
            print(f"U_x: {U_x}")
            U_xx = tape.gradient(U_x, x)
        return U, U_x, U_xx

    @tf.function
    def net_U1(self, x):
        lambda_1 = self.lambda_1
        lambda_2 = tf.exp(self.lambda_2)
        U = self.model(x)
        with tf.GradientTape(persistent=True) as tape1:
            tape1.watch(x)
            U_x = tape1.gradient(U, x)
        U_xx = tape1.gradient(U_x, x)
        U_xxx = tape1.gradient(U_xx, x)
        tape1.reset()
        F = -lambda_1*U*U_x - lambda_2*U_xxx
        U1 = U + self.dt*tf.matmul(F, tf.transpose((self.IRK_beta - self.IRK_alpha)))
        return U1



    @tf.function
    def call(self, x0, x1, u0, u1):
        U0_pred = self.net_U0(x0)
        U1_pred = self.net_U1(x1)
        loss = tf.reduce_sum(tf.square(u0 - U0_pred)) + \
               tf.reduce_sum(tf.square(u1 - U1_pred))
        return loss

    def train(self, nIter):
        optimizer_Adam = tf.keras.optimizers.Adam()
        for it in range(nIter):
            with tf.GradientTape() as tape:
                loss_value = self.call(self.x0, self.x1, self.u0, self.u1)
            grads = tape.gradient(loss_value, self.trainable_variables)
            optimizer_Adam.apply_gradients(zip(grads, self.trainable_variables))

            # Print
            if it % 10 == 0:
                lambda_1_value = self.lambda_1.numpy()
                lambda_2_value = np.exp(self.lambda_2.numpy())
                print('It: %d, Loss: %.3e, l1: %.3f, l2: %.5f' % 
                      (it, loss_value, lambda_1_value, lambda_2_value))

        # Using L-BFGS-B optimizer from tensorflow_probability after Adam optimizer
        def loss_fn():
            return self.call(self.x0, self.x1, self.u0, self.u1)

        results = tfp.optimizer.lbfgs_minimize(
            loss_fn,
            initial_position=self.trainable_variables,
            max_iterations=50000,
            tolerance=1.0 * np.finfo(float).eps)

        # Assign the optimization results to model parameters
        var_updates = []
        for var, new_var in zip(self.trainable_variables, results.position):
            var_updates.append(var.assign(new_var))
        self.model.updates = var_updates

    def predict(self, x_star):
        U0_star = self.net_U0(x_star)
        U1_star = self.net_U1(x_star)
        return U0_star, U1_star


In [167]:
import scipy

In [168]:
q = 50
skip = 120

N0 = 199
N1 = 201
layers = [1, 50, 50, 50, 50, q]

data = scipy.io.loadmat('./PINNs-master/main/Data/KdV.mat')

t_star = data['tt'].flatten()[:,None]
x_star = data['x'].flatten()[:,None]
Exact = np.real(data['uu'])

idx_t = 40

In [169]:
noise = 0.0    

idx_x = np.random.choice(Exact.shape[0], N0, replace=False)
x0 = x_star[idx_x,:]
u0 = Exact[idx_x,idx_t][:,None]
u0 = u0 + noise*np.std(u0)*np.random.randn(u0.shape[0], u0.shape[1])
    
idx_x = np.random.choice(Exact.shape[0], N1, replace=False)
x1 = x_star[idx_x,:]
u1 = Exact[idx_x,idx_t + skip][:,None]
u1 = u1 + noise*np.std(u1)*np.random.randn(u1.shape[0], u1.shape[1])

dt = (t_star[idx_t+skip] - t_star[idx_t]).item()  

In [170]:
lb = x_star.min(0)
ub = x_star.max(0)

model = PhysicsInformedNN(x0, u0, x1, u1, layers, dt, lb, ub, q)

In [ ]:
model.train(nIter = 50000)

In [ ]:
U0_pred, U1_pred = model.predict(x_star) 